In [3]:
import pandas as pd
import pycaret.classification as pyc
import numpy as np
from sklearn.model_selection import train_test_split
import random
import ast
import pydash
import numpy as np

import sys
sys.path.append('../../../scripts/')
import PreparaDatasetV3 as v3
import CreationFuncs as cf

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [4]:
TARGET = 'Back_Home'
ODDS = 'Odds_H'
SPREAD = 1
TYPE_BET = 'back'

In [5]:
cf.setup_model_params(target = TARGET, odds = ODDS, type_bet = TYPE_BET, spread = SPREAD)

# Carregando o dataset

In [6]:
data = pd.read_csv('../../../data/entire_season.csv')
data = v3.prepara_df(data)

In [7]:
def prepara_dataset(df):
    df = df.query("1.3 <= Odds_H <= 2")
    df = df.query("League == 'Eua Nba'")

    # Limpa df
    df.replace([np.inf, -np.inf], 0, inplace=True)
    # df.dropna(subset=['Avg_CG_H', 'Avg_CG_A', TARGET], inplace=True)

    df.reset_index(drop=True, inplace=True)

    return df

In [8]:
df = data.copy()
df = prepara_dataset(df)
df['Date'] = pd.to_datetime(df['Date'])

In [9]:
# DIA: 18/01/2025
df.query('Date < "2025-01-18"', inplace=True)
print(f'{df.shape[0]} jogos no dataset')
print(f'Win rate: {df[TARGET].mean() * 100:.2f}% - Odd relativa: {1/df[TARGET].mean():.2f}')
print(f'Odd real: {df[ODDS].mean():.2f}')

3241 jogos no dataset
Win rate: 60.63% - Odd relativa: 1.65
Odd real: 1.59


Criar os blocos de n jogos (se não for usar clusterização)

In [ ]:
# blocks of n rows
dfs = cf.create_batches(df, 800)
treinamento = pd.concat([dfs[0], dfs[1]])
entire_val = pd.concat(dfs[2:], ignore_index=True)
val = dfs[2]
print(f'{len(dfs)} blocos')

# Criação do modelo de cluster

In [22]:
features = ['Over_Line', 'HA_Line', 'p_H', 'p_A', 'H_A', 'A_H', 'H_O','A_O','Euc_Dist_MO', 'Angle_MO', 'Euc_Dist_OV',
 'Angle_OV', 'DifAbs_HomeAway', 'Angle_HomeAway', 'DifPer_HomeAway', 'Media_Ptos_H', 'Media_Ptos_A', 'DesvPad_Ptos_H', 'DesvPad_Ptos_A',
 'CV_Ptos_H', 'CV_Ptos_A', ODDS]

In [23]:
from pycaret.clustering import *

stp = setup(data = df, ignore_features = [x for x in df.columns.to_list() if x not in features], verbose = False, session_id = 2024, memory = False)

In [ ]:
c_model = create_model('kmeans', 4)

In [ ]:
clustered = assign_model(c_model)
clustered = pd.merge(df[[TARGET, 'Date', 'Home', 'Away', 'Home_Pts', 'Away_Pts']], clustered, left_index=True, right_index=True)

# Separa jogos aleatoriamente em treinamento e validação
treinamento, entire_val = train_test_split(clustered, train_size=0.40, random_state=2024)

# Cria blocos de jogos
dfs = cf.create_batches(entire_val, 500)
print(f'{len(dfs)} dataframes\n')

print(f'Treino: {treinamento.shape[0]}\n')

c0 = treinamento[treinamento['Cluster'] == 'Cluster 0']
c1 = treinamento[treinamento['Cluster'] == 'Cluster 1']
c2 = treinamento[treinamento['Cluster'] == 'Cluster 2']
c3 = treinamento[treinamento['Cluster'] == 'Cluster 3']
c4 = treinamento[treinamento['Cluster'] == 'Cluster 4']
c5 = treinamento[treinamento['Cluster'] == 'Cluster 5']
c6 = treinamento[treinamento['Cluster'] == 'Cluster 6']
c7 = treinamento[treinamento['Cluster'] == 'Cluster 7']

print(f'Data normal - Média WR: {treinamento[TARGET].mean() * 100:.2f}%')
print(f'Cluster 0 - Média WR: {c0[TARGET].mean() * 100:.2f}% ({c0.shape[0]} jogos)')
print(f'Cluster 1 - Média WR: {c1[TARGET].mean() * 100:.2f}% ({c1.shape[0]} jogos)')
print(f'Cluster 2 - Média WR: {c2[TARGET].mean() * 100:.2f}% ({c2.shape[0]} jogos)')
print(f'Cluster 3 - Média WR: {c3[TARGET].mean() * 100:.2f}% ({c3.shape[0]} jogos)')
print(f'Cluster 4 - Média WR: {c4[TARGET].mean() * 100:.2f}% ({c4.shape[0]} jogos)')
print(f'Cluster 5 - Média WR: {c5[TARGET].mean() * 100:.2f}% ({c5.shape[0]} jogos)')
print(f'Cluster 6 - Média WR: {c6[TARGET].mean() * 100:.2f}% ({c6.shape[0]} jogos)')
print(f'Cluster 7 - Média WR: {c7[TARGET].mean() * 100:.2f}% ({c7.shape[0]} jogos)')

Analisa resultados de profit nos clusters

In [ ]:
# c_profit = clustered.copy()
# c_profit.query("Cluster == 'Cluster 3'", inplace=True)
# c_profit.loc[(c_profit[TARGET] == 0), 'Profit'] = 0.94
# c_profit.loc[(c_profit[TARGET] == 1), 'Profit'] = - c_profit[ODDS] - 1
# c_profit['Acumulado'] = c_profit['Profit'].cumsum()
# c_profit_value = c_profit['Acumulado'].iloc[-1]

# print(f'Profit: {c_profit_value:.2f}')
# print(f'ROI: {c_profit_value / len(c_profit):.2f}')

# c_profit[[ODDS]].describe()

Seleciona o cluster

In [ ]:
CHOSEN_CLUSTER = 'Cluster 1'

treinamento.query('Cluster == @CHOSEN_CLUSTER or Cluster == "Cluster 3" or Cluster == "Cluster 4" or Cluster == "Cluster 5"', inplace=True)
entire_val.query('Cluster == @CHOSEN_CLUSTER or Cluster == "Cluster 3" or Cluster == "Cluster 4" or Cluster == "Cluster 5"', inplace=True)

for df in dfs:
    df.query('Cluster == @CHOSEN_CLUSTER or Cluster == "Cluster 3" or Cluster == "Cluster 4" or Cluster == "Cluster 5"', inplace=True)

val = dfs[0].copy()

print(f'Treino: {treinamento.shape[0]}')

In [ ]:
save_model(c_model, 'cluster')

# Seleção de variáveis para o modelo

In [12]:
features = ['FT_Odds_H', 'FT_Odds_D', 'FT_Odds_A', 'Odds_O25', 'Odds_U25', 'BTTS_Yes', 'BTTS_No', 'CV_Odds', 'H_D',
 'H_A', 'D_H', 'D_A', 'A_H', 'A_D', 'O_D', 'O_S', 'H_O', 'A_O', 'Odds_Sum', 'HO_D', 'AO_D', 'ADJ_H', 'ADJ_A', 'Euc_Dist_MO',
 'Angle_MO', 'Euc_Dist_OV', 'Angle_OV', 'DifAbs_HomeAway', 'DifAbs_HomeDraw', 'DifAbs_DrawAway', 'Angle_HomeAway',
 'Angle_HomeDraw', 'Angle_DrawAway', 'DifPer_HomeAway', 'DifPer_HomeDraw', 'DifPer_DrawAway', ODDS]


### Variáveis escolhidas aleatóriamente

In [13]:
colunas = features
model_name = 'nb'

In [ ]:
melhor_PL = -10000

for i in range(30):
    feats = cf.random_variables(var_list=colunas, min_vars=5, max_vars=15)
    feats.extend([ODDS])

    metrics = cf.create_model_sample(data = treinamento, fts = feats, seed=300, model_algorithm = model_name, fix_imbalance=True)

    print(f'Rodada {i+1}')

    if metrics['pl'] > melhor_PL:
        melhor_PL = metrics['pl']
        print(f"**Novo melhor**")
        print(f"Precision: {metrics['prec_model']:.4f} | STD: {metrics['std_model']:.2f}")
        print(f"PL: {metrics['pl']:.2f} | ROI: {100*metrics['roi']:.2f}% | EM: {metrics['ev']:.2f} | Prec.: {metrics['wr']:.2f} | Entradas: {metrics['porc_ent']:.2f}")
        print(f"features = {feats}\n")
    print()

### Redução de variáveis

In [37]:
fits = ['FT_Odds_D', 'H_D', 'Odds', 'D_H', 'Angle_HomeAway', 'DifAbs_DrawAway', 'ADJ_A', 'Euc_Dist_MO', 'BTTS_Yes', 'DifAbs_HomeAway', 'H_A', 'BTTS_No', 'DifAbs_HomeDraw', 'Odds']
fits = ['FT_Odds_D', 'H_D', 'Odds', 'D_H', 'DifAbs_DrawAway', 'ADJ_A', 'Euc_Dist_MO', 'BTTS_Yes', 'DifAbs_HomeAway', 'H_A', 'BTTS_No', 'DifAbs_HomeDraw', 'Odds']
fits = ['FT_Odds_D', 'H_D', 'Odds', 'D_H', 'DifAbs_DrawAway', 'ADJ_A', 'BTTS_Yes', 'H_A', 'BTTS_No', 'DifAbs_HomeDraw', 'Odds']
fits = ['FT_Odds_D', 'Odds', 'D_H', 'DifAbs_DrawAway', 'ADJ_A', 'BTTS_Yes', 'H_A', 'BTTS_No', 'DifAbs_HomeDraw', 'Odds']
fits = ['FT_Odds_D', 'Odds', 'D_H', 'DifAbs_DrawAway', 'BTTS_Yes', 'H_A', 'BTTS_No', 'DifAbs_HomeDraw', 'Odds']


In [ ]:
metrics = cf.create_model_sample(data = treinamento, fts = fits, seed=300, model_algorithm = model_name, fix_imbalance=True)

print('---------------------- BASE ----------------------')
print(f'Precision: {metrics["prec_model"]:.4f} | Std: {metrics["std_model"]:.2f}')
print(f'PL: {metrics["pl"]:.2f} | ROI: {100*metrics["roi"]:.2f}% | EM: {metrics["ev"]:.2f} | Prec.: {metrics["wr"]:.2f} | Entradas: {metrics["porc_ent"]:.2f}')
print(f'fits = {fits}\n')
print('--------------------------------------------------')
melhor_PL = metrics["pl"]

for i in range(len(fits) - 1):
    feat = fits[i]
    fits.pop(i)
    print(f'Removido: {feat}')

    metrics = cf.create_model_sample(data = treinamento, fts = fits, seed=300, model_algorithm = model_name, fix_imbalance=True)

    if metrics["pl"] >= melhor_PL:
        print('*** NOVO MELHOR ***')
        melhor_PL = metrics["pl"]
    print(f'Precision: {metrics["prec_model"]:.4f} | Std: {metrics["std_model"]:.2f}')
    print(f'PL: {metrics["pl"]:.2f} | ROI: {100*metrics["roi"]:.2f}% | EM: {metrics["ev"]:.2f} | Prec.: {metrics["wr"]:.2f} | Entradas: {metrics["porc_ent"]:.2f}')
    print(f'fits = {fits}\n')
    fits.insert(i, feat)
    print()

# Cross validation manual para teste de seeds

In [39]:
features = ['FT_Odds_H', 'FT_Odds_D', 'FT_Odds_A', 'Odds_O25', 'Odds_U25', 'BTTS_Yes', 'BTTS_No', 'CV_Odds', 'H_D',
 'H_A', 'D_H', 'D_A', 'A_H', 'A_D', 'O_D', 'O_S', 'H_O', 'A_O', 'Odds_Sum', 'HO_D', 'AO_D', 'ADJ_H', 'ADJ_A', 'Euc_Dist_MO',
 'Angle_MO', 'Euc_Dist_OV', 'Angle_OV', 'DifAbs_HomeAway', 'DifAbs_HomeDraw', 'DifAbs_DrawAway', 'Angle_HomeAway',
 'Angle_HomeDraw', 'Angle_DrawAway', 'DifPer_HomeAway', 'DifPer_HomeDraw', 'DifPer_DrawAway', 'Media_Ptos_H', 'Media_Ptos_A',
 'DesvPad_Ptos_H', 'DesvPad_Ptos_A', 'CV_Ptos_H', 'CV_Ptos_A', 'DifPtos_H', 'Media_GM_H', 'Media_GM_A', 'DesvPad_GM_H', 'DesvPad_GM_A',
 'CV_GM_H', 'CV_GM_A', 'DifGM_H', 'Media_GS_H', 'Media_GS_A', 'DesvPad_GS_H', 'DesvPad_GS_A', 'CV_GS_H', 'CV_GS_A', 'DifGS_H',
 'Media_SG_H', 'Media_SG_A', 'DesvPad_SG_H', 'DesvPad_SG_A', 'CV_SG_H', 'CV_SG_A', 'DifSG_H', 'Media_SG_H_01', 'Media_SG_A_01', 'DesvPad_SG_H_01',
 'DesvPad_SG_A_01', 'CV_SG_H_01', 'CV_SG_A_01', 'Media_SG_H_02', 'Media_SG_A_02', 'DesvPad_SG_H_02', 'DesvPad_SG_A_02', 'CV_SG_H_02',
 'CV_SG_A_02', 'Media_CG_H_02', 'Media_CG_A_02', 'DesvPad_CG_H_02', 'DesvPad_CG_A_02', 'CV_CG_H_02', 'CV_CG_A_02', 'Dif_CG_02', 'MediaOddsHome',
 'MediaOddsAway', 'OMQB', 'xJoneHome2', 'xJoneAway2', 'Media_0x0_Home', 'Media_0x0_Away', 'Media_Goals_Home', 'Media_Goals_Away', 'Media_0G_Home',
 'Media_0G_Away', ODDS]

fits = ['FT_Odds_D', 'Odds', 'D_H', 'DifAbs_DrawAway', 'BTTS_Yes', 'H_A', 'BTTS_No', 'DifAbs_HomeDraw', 'Odds']

In [ ]:
model_name = 'nb'

seeds = random.sample(range(0, 9999), 10)

for i in range(len(seeds)):
        metrics = cf.create_model_sample(data = treinamento, fts = features, seed=seeds[i], model_algorithm = model_name, fix_imbalance = True)

        previsoes = metrics['dataframe']
        pl = metrics['pl']
        roi = metrics['roi']
        oddback = metrics['oddback']
        ev = metrics['ev']
        wr = metrics['wr']
        porc_ent = metrics['porc_ent']

        print(f'Fold {i+1} (seed {seeds[i]})')
        print(f'PL: {pl:.2f} | ROI: {100*roi:.2f}% | Odd média: {oddback:.2f} | EM: {ev:.2f} | Prec.: {wr:.2f} | Entradas: {porc_ent:.2f}')
        cf.plot_mini_chart(previsoes)
        print()

In [ ]:
2235
7741

# Criação do modelo de classificação

In [11]:
features = [ 'Over_Line', 'HA_Line', 'p_H', 'p_A', 'H_A', 'A_H', 'H_O','A_O','Euc_Dist_MO', 'Angle_MO', 'Euc_Dist_OV',
 'Angle_OV', 'DifAbs_HomeAway', 'Angle_HomeAway', 'DifPer_HomeAway', 'Media_Ptos_H', 'Media_Ptos_A', 'DesvPad_Ptos_H', 'DesvPad_Ptos_A',
 'CV_Ptos_H', 'CV_Ptos_A', 'DifPtos_H', 'Media_GM_H', 'Media_GM_A', 'DesvPad_GM_H', 'DesvPad_GM_A', 'CV_GM_H', 'CV_GM_A',
 'DifGM_H', 'Media_GS_H', 'Media_GS_A', 'DesvPad_GS_H', 'DesvPad_GS_A', 'CV_GS_H', 'CV_GS_A', 'DifGS_H', 'Media_SG_H', 'Media_SG_A',
 'DesvPad_SG_H', 'DesvPad_SG_A', 'CV_SG_H', 'CV_SG_A', 'DifSG_H', 'Media_SG_H_01', 'Media_SG_A_01', 'DesvPad_SG_H_01', 'DesvPad_SG_A_01',
 'CV_SG_H_01', 'CV_SG_A_01', 'Media_SG_H_02', 'Media_SG_A_02', 'DesvPad_SG_H_02', 'DesvPad_SG_A_02', 'CV_SG_H_02', 'CV_SG_A_02',
 'Media_CG_H_02', 'Media_CG_A_02', 'DesvPad_CG_H_02', 'DesvPad_CG_A_02', 'CV_CG_H_02', 'CV_CG_A_02', 'Dif_CG_02', 'MediaOddsHome', 'MediaOddsAway',
 'Media_Goals_Home', 'Media_Goals_Away', ODDS]

In [12]:
############ lembrar que adicionei o fix imbalance = true
real_treino, real_teste = cf.setup_pycaret(data = treinamento, features = features, seed = 300, fix_imbalance=False)

In [ ]:
top = pyc.compare_models(exclude=['catboost', 'dummy', 'lightgbm'], fold = 30, sort='precision', n_select=14)

In [ ]:
model = pyc.create_model('nb', fold = 5, verbose=True)

In [15]:
def filtra_resultados(_df, filter = 0):
    _df = _df[_df['prediction_label'] == 1]

    # if filter == 1:
    #     _df = _df.query("2.791 < FT_Odds_D <= 3.288")

    return _df

In [ ]:
mdl = model
flt = 1

print('TREINO', '-'*80)
df_treino = cf.info_model(mdl, real_treino, filtra_resultados, filter = flt)
print('TESTE', '-'*80)
df_teste = cf.info_model(mdl, real_teste, filtra_resultados, filter = flt)
# print('TREINO & TESTE', '-'*80)
# df_treinamento = cf.info_model(mdl, treinamento, filtra_resultados, filter = flt)
print('VALIDAÇÃO', '-'*80)
df_val = cf.info_model(mdl, val, filtra_resultados, filter = flt)

### Teste em todos os dfs de validação

In [ ]:
for i in range(len(dfs)):
    aux_dfs = cf.info_model(mdl, dfs[i], filtra_resultados, filter = flt)

### Completo

In [ ]:
df_treinamento_aux = cf.info_model(mdl, treinamento, filtra_resultados, filter = flt, show_info=False)
df_val_aux = cf.info_model(mdl, entire_val, filtra_resultados, filter = flt, show_info=False)
completo = pd.concat([df_treinamento_aux, df_val_aux])
completo.reset_index(inplace=True, drop=True)

print('COMPLETO ORDENADO POR DATA', '-'*80)
df_df = cf.info_model(mdl, df, filtra_resultados, filter = flt)

print('COMPLETO ORDENADO POR TREINO + VALIDAÇÃO', '-'*80)

cf.plot_chart(completo, line_limit=len(df_treinamento_aux))


### Entire Val

In [ ]:
df_entire_val = cf.info_model(mdl, entire_val, filtra_resultados, filter = flt)

### Teste Val

In [ ]:
teste_backup = real_teste.copy()
teste_backup = teste_backup.merge(df[['Date', 'Home', 'Away', 'FTHG', 'FTAG']], how='left', left_index=True, right_index=True)
teste_val = pd.concat([teste_backup, entire_val]).sort_values(by='Date').reset_index(drop=True)
print('TESTE E VALIDAÇÃO', '-'*80)
df_teste_val = cf.info_model(mdl, teste_val, filtra_resultados, filter = flt)

### Salvar modelo e histórico de validação

In [ ]:
# cf.export_val(df_teste_val)
pyc.save_model(model, 'v2_lay_goals_away')

# QCut automático

In [39]:
autocut_treino = df_treino.copy()
autocut_teste = df_teste.copy()
autocut_val = df_val.copy()

In [40]:
min_bins = 3
max_bins = 15
metrica = ['wr', 'pl']

In [41]:
def test_bins(train_data, test_data, val_data, features, bins, n_ranges, metric, limit, min_entries=0.02):
    variaveis = features.copy()
    ranges_dict = {}
    filtered_ranges = []
    string_list = []

    for var in variaveis:
        try:
            new_dict = cf.find_top_range(var, n_ranges = n_ranges, treino = train_data, teste = test_data, validacao = val_data, bins = bins, target = 'Profit')
            ranges_dict[var] = new_dict
        except:
            pass

    ordenado = {chave: valor for chave, valor in sorted(ranges_dict.items(), key=lambda item: item[1]['metric_val'][metric[0]], reverse=True)}

    for var in ordenado:
        diff_treino = ordenado[var]["metric_treino"][metric[0]] - ordenado[var]["metric_tr_base"][metric[0]]
        diff_teste = ordenado[var]["metric_teste"][metric[0]] - ordenado[var]["metric_te_base"][metric[0]]
        diff_val = ordenado[var]["metric_val"][metric[0]] - ordenado[var]["metric_val_base"][metric[0]]
        entradas = ordenado[var]["metric_val"]["porc_ent"]
        if ((diff_treino > 0) & (diff_teste > 0) & (diff_val > 0) & (entradas > min_entries) & (abs(ordenado[var]["metric_val"][metric[0]] - ordenado[var]["metric_teste"][metric[0]]) < limit)):
            string_list.append(f'{var} | {bins} bins | {n_ranges} ranges')

            print(f'{var} | {bins} bins | {n_ranges} ranges')
            print(f'treino: {ordenado[var]["metric_treino"][metric[0]]:.4f} ({ordenado[var]["metric_treino"][metric[0]] - ordenado[var]["metric_tr_base"][metric[0]]:.2f})   |   {ordenado[var]["metric_treino"][metric[1]]:.2f} ({ordenado[var]["metric_treino"][metric[1]] - ordenado[var]["metric_tr_base"][metric[1]]:.2f})')
            print(f'teste: {ordenado[var]["metric_teste"][metric[0]]:.4f} ({ordenado[var]["metric_teste"][metric[0]] - ordenado[var]["metric_te_base"][metric[0]]:.2f})   |   {ordenado[var]["metric_teste"][metric[1]]:.2f} ({ordenado[var]["metric_teste"][metric[1]] - ordenado[var]["metric_te_base"][metric[1]]:.2f})')
            print(f'val: {ordenado[var]["metric_val"][metric[0]]:.4f} ({ordenado[var]["metric_val"][metric[0]] - ordenado[var]["metric_val_base"][metric[0]]:.2f})   |   {ordenado[var]["metric_val"][metric[1]]:.2f} ({ordenado[var]["metric_val"][metric[1]] - ordenado[var]["metric_val_base"][metric[1]]:.2f})')
            print(f'% Ent: {ordenado[var]["metric_val"]["porc_ent"]:.2f}')
            ordenado[var]['variavel'] = var
            filtered_ranges.append(ordenado[var])
            print()

    return filtered_ranges, string_list

In [ ]:
filtered_dicts = []
filters_string = ''

for bin in range(max_bins - 1, min_bins - 1, -1):
    for _ in range(bin-1, -1, -1):
        ranges, string_list = test_bins(autocut_treino, autocut_teste, autocut_val, features, bins = bin, n_ranges = _ + 1, metric = metrica, limit = 0.01, min_entries=0.50)
        if len(ranges) > 0:
            for str_on_list in string_list:
                filters_string += str_on_list + '\n'
            for i in range(len(ranges)):
                intervalo = ranges[i]['range']
                filtered_dicts.append({'bins': bin, 'n_ranges': _ + 1, 'range': intervalo, 'var': ranges[i]['variavel']})

In [ ]:
# for bin in range(max_bins - 1, min_bins - 1, -1):
#     for _ in range(bin-1, -1, -1):
#         ranges = cf.test_bins(autocut_treino, autocut_teste, autocut_val, features, bins = bin, n_ranges = _ + 1, metric = metrica, limit = 0.02, min_entries=0.5)

In [22]:
def create_query(column, intervals):
    query_parts = [f"{interval.left} < {column} <= {interval.right}" for interval in intervals]
    query_string = " or ".join(query_parts)
    return query_string

def create_filters_object(filters_string):
    result = []
    for line in filters_string.strip().split("\n"):
        parts = line.split(" | ")
        result.append({
            "var": parts[0],
            "bins": int(parts[1].split()[0]),
            "ranges": int(parts[2].split()[0])
        })
    return result

def filter_by_string(dataframe, filter_intervals, use_model=True):
    df = dataframe.copy()
    if use_model:
        df = df[df['prediction_label'] == 1]

    for filter_interval in filter_intervals:
        df = df.query(filter_interval)

    return df

def each_val_df(query_string, num_dfs=0, use_model=True):
    wr_blocks = []

    if num_dfs == 0:
        num_dfs = len(dfs)

    for dataframe in dfs[2:num_dfs]:
        aux_dfs = pyc.predict_model(mdl, dataframe, verbose=False)
        aux_dfs = filter_by_string(aux_dfs, query_string, use_model=use_model)
        aux_dfs = cf.calculate_profit(aux_dfs)
        metrics = cf.calculate_metrics(aux_dfs, total=dataframe.shape[0], return_metrics=True)
        wr_blocks.append(metrics['wr'])

    inf_lim = np.mean(wr_blocks) - 1 * np.std(wr_blocks)
    sup_lim = np.mean(wr_blocks) + 1 * np.std(wr_blocks) if np.mean(wr_blocks) + 1 * np.std(wr_blocks) < 1 else 1

    print(f'Média WR: {np.mean(wr_blocks) * 100 :.2f}')
    print(f'Std: {np.std(wr_blocks) * 100 :.2f}')
    print(f'Margem 1 Std: De {inf_lim * 100 :.2f}% a {sup_lim * 100 :.2f}%')

    return inf_lim

In [ ]:
from IPython.display import display, HTML

filters_from_string = create_filters_object(filters_string)
best_inf_lim = -1000

for f in filters_from_string:
    try:
        print(f['var'], f['bins'], f['ranges'])
        found_range = pydash.find(filtered_dicts, lambda item: item['var'] == f['var'] and item['bins'] == f['bins'] and item['n_ranges'] == f['ranges'])['range']
        q_str = create_query(f['var'], found_range)
        inf_lim = each_val_df([
            '-30.926 < Media_SG_H_01 <= -13.76 or 3.338 < Media_SG_H_01 <= 20.436',
            '0.113 < DifPer_HomeAway <= 0.179 or 0.245 < DifPer_HomeAway <= 0.644',
            '0.356 < H_A <= 0.617 or 0.813 < H_A <= 0.878',
                q_str
            ], num_dfs=10, use_model=True) # Define se o algoritmo vai usar as saídas do modelo ou não
        if inf_lim >= best_inf_lim:
            best_inf_lim = inf_lim
            display(HTML('<p style="font-weight:bold; margin-top:-12px;" >  ⬆️⬆️⬆️ NEW BEST ⬆️⬆️⬆️ </p>'))
        print()
    except:
        print('erro\n')

In [ ]:
chosen_var = 'Media_Goals_Away'
chosen_bins = 8
chosen_n_ranges = 6

query_str = create_query(chosen_var, pydash.find(filtered_dicts, lambda item: item['var'] == chosen_var and item['bins'] == chosen_bins and item['n_ranges'] == chosen_n_ranges)['range'])
query_str

In [ ]:
def filtra_resultados(_df, filtro = 0,):
    _df = _df[_df['prediction_label'] == 1]

    if filtro == 55:
        _df = _df.query("-30.926 < Media_SG_H_01 <= -13.76 or 3.338 < Media_SG_H_01 <= 20.436")
        _df = _df.query("0.113 < DifPer_HomeAway <= 0.179 or 0.245 < DifPer_HomeAway <= 0.644")
        _df = _df.query("0.356 < H_A <= 0.617 or 0.813 < H_A <= 0.878")
        _df = _df.query("89.961 < Media_Goals_Away <= 94.85 or 99.7 < Media_Goals_Away <= 114.25 or 119.1 < Media_Goals_Away <= 128.8")

    return _df

# print('TREINO', '-'*80)
df_treino = cf.info_model(mdl, real_treino, filtra_resultados, filter = 55, show_info=True)
# print('TESTE', '-'*80)
df_teste = cf.info_model(mdl, real_teste, filtra_resultados, filter = 55, show_info=True)
# print('TREINO & TESTE', '-'*80)
# df_treinamento = cf.info_model(mdl, treinamento, filtra_resultados, filter = 55, show_info=True)
# print('VALIDAÇÃO', '-'*80)
df_val = cf.info_model(mdl, val, filtra_resultados, filter = 55, show_info=True)
# df_entire_val = cf.info_model(mdl, entire_val, filtra_resultados, filter = 55, show_info=True)

In [ ]:
import numpy as np
wr_blocks = []

for i_df in dfs:
    aux_dfs = pyc.predict_model(mdl, i_df, verbose=False)
    aux_dfs = filtra_resultados(aux_dfs, filtro = 55)
    aux_dfs = cf.calculate_profit(aux_dfs)
    metrics = cf.calculate_metrics(aux_dfs, total=i_df.shape[0], return_metrics=True)
    wr_blocks.append(metrics['wr'])

print(f'Média WR: {np.mean(wr_blocks) * 100 :.2f}')
print(f'Std: {np.std(wr_blocks) * 100 :.2f}')
print(f'Margem 1 Std: De {(np.mean(wr_blocks) - np.std(wr_blocks)) * 100 :.2f}% a {(np.mean(wr_blocks) + np.std(wr_blocks)) * 100 :.2f}%')

In [ ]:
Média WR: 96.32
Std: 2.68

#### Teste em todos os dfs de validação

In [ ]:
for i in dfs:
    aux_dfs = cf.info_model(mdl, i, filtra_resultados, filter = 0)
    aux_dfs = cf.info_model(mdl, i, filtra_resultados, filter = 55)
    print('----------------------------------------------------')

#### Teste + Validação completa

In [ ]:
teste_backup = real_teste.copy()
teste_backup = teste_backup.merge(df[['Date', 'Home', 'Away', 'FTHG', 'FTAG']], how='left', left_index=True, right_index=True)
teste_val = pd.concat([teste_backup, entire_val]).sort_values(by='Date').reset_index(drop=True)
print('TESTE E VALIDAÇÃO', '-'*80)
df_teste_val = cf.info_model(mdl, teste_val, filtra_resultados, filter = 55)

In [ ]:
print('VALIDAÇÃO TOTAL', '-'*80)
df_entire_val = cf.info_model(mdl, entire_val, filtra_resultados, filter = 55)

In [ ]:
print('COMPLETO ORDENADO POR DATA', '-'*80)
df_df = cf.info_model(mdl, clustered.query('Cluster == @CHOSEN_CLUSTER or Cluster == 2'), filtra_resultados, filter = 55)

# QCut

In [ ]:
df_cut = df_treino.copy()
df_cut2 = df_teste.copy()
df_cut3 = df_val.copy()
bins = 10
limiar = 100
var_n = -1

In [ ]:
variaveis = features.copy()
for var in variaveis:
    try:
        cf.find_best_range(df_cut, df_cut2, df_cut3, var, TARGET, bins, limiar=limiar)
    except:
        continue

In [ ]:
cf.find_best_range(df_cut, df_cut2, df_cut3, 'Prob_1GA', TARGET, bins, limiar=limiar)

In [ ]:
def filtra_resultados(_df, filtro = 0):
    _df = _df[_df['prediction_label'] == 1]
    if filtro == 55:
        _df = _df.query("0.0299 < Prob_1GA <= 0.0898 or 0.15 < Prob_1GA <= 0.239")

    return _df

print('TREINO', '-'*80)
df_treino = cf.info_model(mdl, real_treino, filtra_resultados, filter = 55, show_info=True)
print('TESTE', '-'*80)
df_teste = cf.info_model(mdl, real_teste, filtra_resultados, filter = 55, show_info=True)
print('TREINO & TESTE', '-'*80)
df_treinamento = cf.info_model(mdl, treinamento, filtra_resultados, filter = 55, show_info=False)
print('VALIDAÇÃO', '-'*80)
df_val = cf.info_model(mdl, val, filtra_resultados, filter = 55, show_info=True)
# print('COMPLETO', '-' *80)
# completo = pd.concat([df_teste, df_val])
# cf.calculate_metrics(completo, len(data))
# cf.plot_chart(completo)

In [ ]:
import numpy as np
wr_blocks = []

for i in range(len(dfs)):
    aux_dfs = pyc.predict_model(mdl, dfs[i], verbose=False)
    aux_dfs = filtra_resultados(aux_dfs, filtro = 55)
    aux_dfs = cf.calculate_profit(aux_dfs)
    metrics = cf.calculate_metrics(aux_dfs, total=df.shape[0], return_metrics=True)
    wr_blocks.append(metrics['wr'])

print(f'Média WR: {np.mean(wr_blocks) * 100 :.2f}')
print(f'Std: {np.std(wr_blocks) * 100 :.2f}')

#### Teste em todos os dfs de validação

In [ ]:
for i in range(len(dfs)):
    aux_dfs = cf.info_model(mdl, dfs[i], filtra_resultados, filter = 55)

# Comparação de modelos

In [ ]:
for i, i_model in enumerate(top):
    # try:
        print(f'- {i_model.__class__.__name__} - {i}')
        cf.info_model(i_model, real_teste, filtra_resultados, filter = 1)
        # cf.info_model(i_model, teste_val, filtra_resultados, filter = 1)
        # cf.info_model(i_model, val, filtra_resultados, filter = 1)
        # cf.info_model(i_model, entire_val, filtra_resultados, filter = 55)
    # except:
    #     continue

Análise de blends

In [ ]:
blend_df = cf.info_model(top[1], teste_val, filtra_resultados, filter = 1)
blend_df = cf.info_model(blended, teste_val, filtra_resultados, filter = 1)

In [ ]:
blended = pyc.blend_models([top[1], top[3]], fold = 5, verbose=False, optimize='precision')
stacked = pyc.stack_models([top[1], top[3]], fold = 5, verbose=False, optimize='precision')

blend_df = cf.info_model(blended, teste_val, filtra_resultados, filter = 1)
blend_df = cf.info_model(stacked, teste_val, filtra_resultados, filter = 1)